In [ ]:
import requests
import json

API = 'http://localhost:5000'
session = requests.Session()

# Building your own language in python

<p style='text-align: right;'> David De Sousa - WeRiot </p>

# What is a Domain Specific Language (DSL)

# Advantages and Disadvantages

# Why use a DSL 

# Real world DSLs you (may) use every day

* HTML
* CSS
* LaTeX
* SQL
* Jinja

# Sample use case

## Client requests a "sum" endpoint


```python
@app.route('/sum')
def sum():
    # skip validations because reasons
    a = float(request.args.get('param1'))
    b = float(request.args.get('param2'))

    return jsonify(result=a + b)
```

done!

In [ ]:
params = (5, 4)

resp = session.get(
    f'{API}/sum', 
    params={'param1': params[0], 'param2': params[1]},
)
print(resp.json())

# Sample use case

#### "Super awesome, but now I want to sum 3 elements"

* Add support in endpoint
* Tell client to call API twice

```python
@app.route('/sum2')
def sum2():
    # skip validations because reasons
    a = float(request.args.get('param1'))
    b = float(request.args.get('param2'))
    c = 0

    if 'param3' in request.args:
        c = float(request.args.get('param3'))

    return jsonify(result=a + b + c)
```

In [ ]:
params = (5, 4, 7)
payload = {'param1': params[0], 'param2': params[1]}
if len(params) > 2:
    payload['param3'] = params[2]

resp = session.get(
    f'{API}/sum2', 
    params=payload,
)
print(resp.json())

# Sample use case

#### Client wants substraction

#### And division
#### And multiplication
#### All at the same time

# Sample use case

## Let's use json!

34 + (2 * 3)
```json
{
"operator": "+",
"param1": 34,
"param2": {
    "operator": "*",
    "param1": 2,
    "param2": 3 }
}
```

```python
def process_operation(operation):
    operators = {
        '*': operator.mul,
        '/': operator.truediv,
        '+': operator.add,
        '-': operator.sub
    }

    param1 = operation['param1']
    param2 = operation['param2']
    operator_function = operators[operation['operator']]

    if type(param1) is dict:
        param1 = process_operation(param1)

    if type(param2) is dict:
        param2 = process_operation(param2)

    return operator_function(param1, param2)


@app.route('/math')
def math():
    operation = json.loads(request.args.get('operation'))

    return jsonify(result=process_operation(operation))
```

In [ ]:
operation = {  #  (6/3) * (7-2)
    "operator": "*", 
    "param1": {
        "operator": "/", 
        "param1": 6, 
        "param2": 3
     }, 
     "param2": {
         "operator": "-",
         "param1": 7,
         "param2": 2
     }
}

payload = {"operation": json.dumps(operation)}
resp = session.get(
    f'{API}/math', 
    params=payload,
)
print(resp.json())

# Sample use case

* Solved!
* Flexible API
* One call can do all the calculations
* Simple, right?

# Sample use case

`((2 + (3 * 4) - 5) * 6) / 4`

```json
{
    "operator": "/",
    "param1": {
        "operator": "*",
        "param1": {
            "operator": "+",
            "param1": 2,
            "param2": {
                "operator": "-",
                "param1": {
                    "operator": "*",
                    "param1": 3,
                    "param2": 4
                },
                "param2": 5
            }
        },
        "param2": 6
    },
    "param2": 4
}
```

# Client is now crying

#### Could we do better?

Which format would be easier for the user?

How about:

`((2 + (3 * 4) - 5) * 6) / 4`

# Your